In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)
opt["base_path"] = "../"

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
pprint(opt["MODEL"]["FREEZE_NAME"])

[{'backbone': True}, {'encoder': True}]


In [5]:
fix_param = opt['SOLVER'].get('FIX_PARAM',{})
fix_param

{'backbone': True, 'encoder': True}

In [6]:
from trainer import HDecoder_Trainer as Trainer
trainer = Trainer(opt)
trainer.train()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as ../data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to ../data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-------------------------------------------------------
INFO:trainer.default_trainer:Training on rank: 0
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
 

Dataset length: 5400
num of train samples: 2700


INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw1.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.bias, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.dw2.weight, Model Shape: torch.Size([96, 1, 3, 3]) <-> Ckpt Shape: torch.Size([96, 1, 3, 3])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_1, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.gamma_2, Model Shape: torch.Size([96]) <-> Ckpt Shape: torch.Size([96])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.bias, Model Shape: torch.Size([384]) <-> Ckpt Shape: torch.Size([384])
INFO:utils.model:Loaded backbone.layers.0.blocks.0.mlp.fc1.weight, Model Shape: torch.Si

Evaluation start ...


INFO:detectron2.data.common:Serializing 4946 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 3.27 MiB
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/4946. Dataloading: 0.0146 s/iter. Inference: 0.2146 s/iter. Eval: 0.0004 s/iter. Total: 0.2297 s/iter. ETA=0:18:53
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 33/4946. Dataloading: 0.0168 s/iter. Inference: 0.2131 s/iter. Eval: 0.0004 s/iter. Total: 0.2303 s/iter. ETA=0:18:51
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 54/4946. Dataloading: 0.0172 s/iter. Inference: 0.2163 s/iter. Eval: 0.0004 s/iter. Total: 0.2340 s/iter. ETA=0:19:04
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 76/4946. Dataloading: 0.0175 s/iter. Inference: 0.2155 s/iter. Eval: 0.0004 s/iter. Total: 0.2334 s/iter. ETA=0:18:56
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 98/4946. Dataloading: 0.0174 s/iter. Inference: 0.2151 s/iter. Eval: 0.0004 s/iter. Total:

...

INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/4946. Dataloading: 0.0145 s/iter. Inference: 0.2157 s/iter. Eval: 0.0004 s/iter. Total: 0.2306 s/iter. ETA=0:18:57
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 33/4946. Dataloading: 0.0164 s/iter. Inference: 0.2141 s/iter. Eval: 0.0004 s/iter. Total: 0.2309 s/iter. ETA=0:18:54
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 54/4946. Dataloading: 0.0170 s/iter. Inference: 0.2175 s/iter. Eval: 0.0004 s/iter. Total: 0.2349 s/iter. ETA=0:19:09
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 76/4946. Dataloading: 0.0170 s/iter. Inference: 0.2166 s/iter. Eval: 0.0004 s/iter. Total: 0.2340 s/iter. ETA=0:18:59
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 98/4946. Dataloading: 0.0171 s/iter. Inference: 0.2161 s/iter. Eval: 0.0004 s/iter. Total: 0.2337 s/iter. ETA=0:18:52
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 120/4946. Dataloading: 0.0171 s/iter. Inference: 0.2153 s/iter. Eval: 0.0004 s/i

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.0455
                  stand: #GTs = 4118, AP = 0.0313
              sit_instr: #GTs = 1916, AP = 0.0004
             ride_instr: #GTs = 0556, AP = 0.0000
                   walk: #GTs = 0597, AP = 0.0023
               look_obj: #GTs = 3347, AP = 0.0004
              hit_instr: #GTs = 0349, AP = 0.0000
                hit_obj: #GTs = 0349, AP = 0.0016
                eat_obj: #GTs = 0521, AP = 0.0000
              eat_instr: #GTs = 0521, AP = 0.0000
             jump_instr: #GTs = 0635, AP = 0.0004
              lay_instr: #GTs = 0387, AP = 0.0000
    talk_on_phone_instr: #GTs = 0285, AP = 0.0001
              carry_obj: #GTs = 0472, AP = 0.0000
              throw_obj: #GTs = 0244, AP = 0.0000
              catch_obj: #GTs = 0246, AP = 0.0004
              cut_instr: #GTs = 0269, AP = 0.0000
                cut_obj: #GTs = 0269, AP = 0.0000
                    run: #GTs = 0687, A

INFO:trainer.default_trainer:{'vcoco_val/vcoco': {'AP_hold_obj': 0.045454545454545456, 'AP_stand': 0.031294626743875835, 'AP_sit_instr': 0.0004340277777777778, 'AP_ride_instr': 0.0, 'AP_walk': 0.002253944402704733, 'AP_look_obj': 0.00036842590034079397, 'AP_hit_instr': 0.0, 'AP_hit_obj': 0.001594896331738437, 'AP_eat_obj': 0.0, 'AP_eat_instr': 0, 'AP_jump_instr': 0.00035790980672870435, 'AP_lay_instr': 0.0, 'AP_talk_on_phone_instr': 0.00011032656663724624, 'AP_carry_obj': 0.0, 'AP_throw_obj': 0.0, 'AP_catch_obj': 0.00043290043290043295, 'AP_cut_instr': 0.0, 'AP_cut_obj': 0.0, 'AP_run': 0.008264462809917356, 'AP_work_on_computer_instr': 0.00037800037800037804, 'AP_ski_instr': 4.2756603757450334e-05, 'AP_surf_instr': 0.00010645092612305727, 'AP_skateboard_instr': 0.00016409583196586809, 'AP_smile': 0.0303030303030303, 'AP_drink_instr': 0, 'AP_kick_obj': 0.0, 'AP_point_instr': 0, 'AP_read_obj': 0, 'AP_snowboard_instr': 0.002840909090909091, 'mAP_all': 0.00428970032279148, 'mAP_thesis': 0.

KeyboardInterrupt: 

: 